### Student Information
Name: 彭星樺

Student ID: 113065507

GitHub ID: ktpss97094

Kaggle name: ktpss97094

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission **BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__.

## config

In [1]:
!pip install transformers[torch] datasets accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.1

In [2]:
### Begin Assignment Here
import os
from enum import Enum
import copy
import torch
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import string
from nltk.stem.porter import PorterStemmer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
import torch
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
from datasets import Dataset
import numpy as np

class EXEC_ENV_ENUM(Enum):
    COLAB = 1
    KAGGLE = 2
    LOCAL = 3

'''
parameters
'''
EXEC_ENV = EXEC_ENV_ENUM.COLAB
# EXEC_ENV = EXEC_ENV_ENUM.KAGGLE
# EXEC_ENV = EXEC_ENV_ENUM.LOCAL
FROM_CHECKPOINT = False
TRAIN = True

input_path = ""
if EXEC_ENV == EXEC_ENV_ENUM.COLAB:
    from google.colab import drive
    drive.mount("/content/gdrive")
    input_path = "/content/gdrive/MyDrive/DMLab2/DM2024-Lab2-Homework"
elif EXEC_ENV == EXEC_ENV_ENUM.KAGGLE:
    input_path = "/kaggle/input/dm-2024-isa-5810-lab-2-homework"
    os.chdir("/kaggle/working")
else:
    input_path = "."
if FROM_CHECKPOINT:
    model_ckpt = os.path.join(input_path, "checkpoint", "best")
else:
    model_ckpt = "distilbert-base-uncased"

SEED = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EMOTIONS = ["anger", "anticipation", "disgust", "fear", "sadness", "surprise", "trust", "joy"]
NUM_EMOTIONS = len(EMOTIONS)
ID_TO_EMOTIONS = {str(index): emotion for index, emotion in enumerate(EMOTIONS)}
EMOTIONS_TO_ID = {emotion: index for index, emotion in enumerate(EMOTIONS)}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/gdrive


In [3]:
def remove_stopwords(text: str) -> str:
    '''
    E.g.,
        text: 'Here is a dog.'
        preprocessed_text: 'Here dog.'
    '''
    stop_word_list = stopwords.words('english')
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token.lower() not in stop_word_list]
    preprocessed_text = ' '.join(filtered_tokens)

    return preprocessed_text


def preprocessing_function(text: str) -> str:
    preprocessed_text = remove_stopwords(text)

    # 轉小寫
    preprocessed_text = preprocessed_text.lower()

    # 移除無用字元
    preprocessed_text = preprocessed_text.replace('<LH>', ' ').replace('<lh>', ' ')

    # 移除 tag (@) 人
    preprocessed_text = re.sub(r"@\w+\s*", "", preprocessed_text)

    # 移除punctuation
    preprocessed_text = "".join([char for char in preprocessed_text if char not in string.punctuation])

    # stemming
    stemmer = PorterStemmer()
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(preprocessed_text)
    stemming_tokens = [stemmer.stem(token) for token in tokens]
    preprocessed_text = ' '.join(stemming_tokens)

    return preprocessed_text

def tokenize(batch, tokenizer):
    return tokenizer(batch["text"], padding=True, truncation=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def draw_plot(log_history):
    training_loss = []
    validation_loss = []
    for log in log_history:
        if "loss" in log:
            training_loss.append(log["loss"])
        if "eval_loss" in log:
            validation_loss.append(log["eval_loss"])

    plt.figure(figsize=(10, 6))
    plt.plot(training_loss, label="Training Loss")
    plt.plot(validation_loss, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.xticks(range(1, len(training_loss) + 1))  # x 軸從 1 開始
    plt.show()
    plt.savefig(os.path.join(input_path, 'loss_plot.png'))

## Preprocessing

In [4]:
preprocessing_train_path = os.path.join(input_path, "preprocessing_train.parquet")
preprocessing_test_path = os.path.join(input_path, "preprocessing_test.parquet")

distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

if os.path.exists(preprocessing_train_path) and os.path.exists(preprocessing_test_path):
    train = pd.read_parquet(preprocessing_train_path)
    test = pd.read_parquet(preprocessing_test_path)
else:
    df = pd.read_json(os.path.join(input_path, "tweets_DM.json"), lines=True)

    # 取出 _source 中的資料
    df["hashtags"] = df["_source"].apply(lambda x: x["tweet"]["hashtags"])
    df["tweet_id"] = df["_source"].apply(lambda x: x["tweet"]["tweet_id"])
    df["text"] = df["_source"].apply(lambda x: x["tweet"]["text"])

    # 將 tweet_id 移到第一個 column
    cols = list(df.columns)
    cols.insert(0, cols.pop(cols.index('tweet_id')))
    df = df[cols]

    # 移除無用 columns
    df = df.drop(columns=["_index", "_crawldate", "_type", "_source"], errors="ignore")

    # 匯入 emotion
    df = df.merge(pd.read_csv(os.path.join(input_path, "emotion.csv")), on="tweet_id", how="left")

    # 分離 train set / test set
    df = df.merge(pd.read_csv(os.path.join(input_path, "data_identification.csv")), on="tweet_id", how="left")
    train = df[df["identification"] == "train"]
    test = df[df["identification"] == "test"]
    train = train.drop(columns=["identification"], errors="ignore")
    test = test.drop(columns=["identification"], errors="ignore")

    # 將 emotion label 編號
    train['label'] = train['emotion'].map(EMOTIONS_TO_ID)
    test['label'] = EMOTIONS_TO_ID["surprise"]  # 隨便一個在範圍內的

    # 做 text 的 preprocessing
    train.loc[:, "text"] = train.loc[:, "text"].apply(preprocessing_function)
    test.loc[:, "text"] = test.loc[:, "text"].apply(preprocessing_function)

    # 用 Distilbert 對 text 做 encode
    train_encoded = train.apply(tokenize, args=(distilbert_tokenizer,), axis=1)
    train['input_ids'] = train_encoded.apply(lambda x: x["input_ids"])
    train['attention_mask'] = train_encoded.apply(lambda x: x["attention_mask"])
    test_encoded = test.apply(tokenize, args=(distilbert_tokenizer,), axis=1)
    test['input_ids'] = test_encoded.apply(lambda x: x["input_ids"])
    test['attention_mask'] = test_encoded.apply(lambda x: x["attention_mask"])

    # 儲存
    train.to_parquet(preprocessing_train_path)
    test.to_parquet(preprocessing_test_path)

# train 時要保留 4 個 column
train = train[['text', 'label', 'input_ids', 'attention_mask']]
test_tweet_id = test["tweet_id"]
test = test[['text', 'label', 'input_ids', 'attention_mask']]

# train set 分割出 validation set
train, validation = train_test_split(train, test_size=0.05, random_state=SEED)

# 轉換為 Hugging Face Dataset
train = Dataset.from_pandas(train)
validation = Dataset.from_pandas(validation)
test = Dataset.from_pandas(test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Fine-Tune

In [ ]:
other_TrainingArguments_args, other_Trainer_args = dict(), dict()

if FROM_CHECKPOINT:
    other_TrainingArguments_args = {"resume_from_checkpoint": model_ckpt}
    other_Trainer_args = {"optimizers": (
                  torch.load(os.path.join(model_ckpt, "optimizer.pt"), map_location=device),
                  torch.load(os.path.join(model_ckpt, "scheduler.pt"), map_location=device)
                  )}

model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=NUM_EMOTIONS, id2label=ID_TO_EMOTIONS, label2id=EMOTIONS_TO_ID)
         .to(device))

batch_size = 64
logging_steps = len(train) // batch_size
model_output_dir = os.path.join(input_path, "checkpoint")
training_args = TrainingArguments(output_dir=model_output_dir,
                                num_train_epochs=10,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                evaluation_strategy="epoch",
                                disable_tqdm=False,
                                logging_steps=logging_steps,
                                log_level="error",
                                save_strategy="epoch",
                                **other_TrainingArguments_args)

trainer = Trainer(model=model, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=train,
                eval_dataset=validation,
                tokenizer=distilbert_tokenizer,
                **other_Trainer_args)

if TRAIN:
    trainer.train()

    draw_plot(trainer.state.log_history)

    validate_pred = trainer.predict(validation)

    display(validate_pred)

test_pred = trainer.predict(test)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-4b3f7cb84a50>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.ru

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.314800,1.243578,0.549911,0.531222


In [ ]:
print(type(model_ckpt))

## Output

In [ ]:
test_pred_label = np.argmax(test_pred.predictions, axis=1)
test_id_label_df = pd.concat([test_tweet_id.rename("id").reset_index(drop=True), pd.DataFrame([ID_TO_EMOTIONS[str(i)] for i in test_pred_label], columns=['emotion'])], axis=1)
sample = pd.read_csv(os.path.join(input_path, "sampleSubmission.csv"))
submission = pd.merge(sample["id"], test_id_label_df, on=["id"])
submission.to_csv(os.path.join(model_output_dir, 'submission.csv'), index=False)

In [ ]:
submission

# Report